In [1]:
from tensorflow.keras.models import load_model

2024-04-09 01:15:19.714729: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-09 01:15:20.298928: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from scipy.optimize import basinhopping

In [3]:
import numpy as np

In [4]:
model = load_model('surogate.h5')

2024-04-09 01:15:25.852365: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9604 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5
2024-04-09 01:15:25.852841: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9621 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


In [14]:
# 特定频率
def loss_freq(s_para):
    # 能量
    E = np.square(s_para)
    E11 = E[0] + E[1]
    E21 = E[2] + E[3]
    E31 = E[4] + E[5]
    E41 = E[6] + E[7]

    # 压制
    loss1 = E11 - E21 - E31 + E41
    # 比例
    loss2 = np.abs(E21 / (E31 + E21) - 0.7)
    # phase
    loss3 = np.abs(np.arctan2(s_para[3], s_para[2]) - np.arctan2(s_para[5], s_para[4]) - np.pi / 4)
    # 能量守恒
    loss4 = np.abs(np.sum(E) - 1)
    
    return loss1 + loss2 + loss3 + loss4

In [59]:
def dgn_obj(s1, s2, s3):
    # return np.max([loss_freq(s1), loss_freq(s2), loss_freq(s3)], axis=0)
    return loss_freq(s1) + loss_freq(s2) + loss_freq(s3)

In [60]:
# 参数数量，迭代次数
NUM_NODES = 1
EPOCHS = 1000

In [61]:
# 物理边界
lb = [1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4]
ub = [5, 5, 5, 5, 5, 5, 5, 5, 100, 100, 100, 100]
bounds = list(zip(lb, ub))

In [62]:
freq = np.array([[2.4], [2.5], [2.6]])

In [63]:
# 定义目标函数
def objective_function(x):
    input = np.concatenate((freq, np.repeat([x], 3, 0)), -1)
    y_pred = model.predict(input, batch_size=3, verbose=0)
    return dgn_obj(y_pred[0], y_pred[1], y_pred[2])

In [64]:
x0 = np.random.uniform(lb, ub)

In [65]:
# 设置basinhopping参数
kwargs = {
    'niter': EPOCHS,  # 总迭代次数
    'T': 1.0,       # 接受概率的温度参数（模拟退火中的温度）
    'stepsize': 0.5, # 步长参数，决定每次尝试跳跃时改变变量的程度
    'disp': True,
    'minimizer_kwargs': {
        'method': 'L-BFGS-B',
        'bounds': bounds
    },  # 局部最小化方法和其参数
}

In [66]:
# 执行basinhopping算法
result = basinhopping(objective_function, x0, **kwargs)

basinhopping step 0: f 14.8264
basinhopping step 1: f 14.8264 trial_f 15.3924 accepted 0  lowest_f 14.8264
basinhopping step 2: f 15.0187 trial_f 15.0187 accepted 1  lowest_f 14.8264
basinhopping step 3: f 15.0187 trial_f 15.5522 accepted 0  lowest_f 14.8264
basinhopping step 4: f 15.4034 trial_f 15.4034 accepted 1  lowest_f 14.8264
basinhopping step 5: f 15.1145 trial_f 15.1145 accepted 1  lowest_f 14.8264
basinhopping step 6: f 16.0245 trial_f 16.0245 accepted 1  lowest_f 14.8264
basinhopping step 7: f 16.1186 trial_f 16.1186 accepted 1  lowest_f 14.8264
basinhopping step 8: f 16.1186 trial_f 16.9453 accepted 0  lowest_f 14.8264
basinhopping step 9: f 15.0012 trial_f 15.0012 accepted 1  lowest_f 14.8264
basinhopping step 10: f 15.0012 trial_f 15.6959 accepted 0  lowest_f 14.8264
basinhopping step 11: f 6.44411 trial_f 6.44411 accepted 1  lowest_f 6.44411
found new global minimum on step 11 with function value 6.44411
basinhopping step 12: f 6.44411 trial_f 7.1464 accepted 0  lowest_f

KeyboardInterrupt: 

In [55]:
# 输出结果
print('全局最小值点:', result.x)
print('全局最小值:', result.fun)

全局最小值点: [  1.           5.           1.           1.           5.
   1.           1.           5.         100.          42.90339342
 100.         100.        ]
全局最小值: 0.00863615075813029
